# How to linux backup

- <a href="https://www.google.com/search?channel=fs&client=ubuntu-sn&q=how+to+linux+backup&bshm=nce/1#fpstate=ive&vld=cid:d262cccb,vid:GqSxR93xK6E">`rsync` usage example</a>;
- <a href="https://download.samba.org/pub/rsync/rsync.1">`rsync` documentation</a>;
- <a href="https://jumpcloud.com/blog/how-to-backup-linux-system-rsync"> backup with `rsync`</a>.

# `rsync` basics

Actually `rsync` is just a copy tool, but if you compare it with `cp` it has much more options and features.

### Basic syntax

`rsync <copied file> <destination of copying>`

Example follows

In [7]:
%%bash
# creating to directories
mkdir dir1 dir2
# creating file in dir1
echo "hello new file" > dir1/test_file
# rsynk file from dir1 to dir2
rsync dir1/test_file dir2/test_file
# check the message
cat dir2/test_file

rm -r dir1 dir2

hello new file


### `r` - copy *r*ecursive

Allow to copy a folder with all its contents.

In the following example I'm trying to copy contents from dir1 to dir2, as you can see I couldn't do it without the `r` option.

In [22]:
%%bash

mkdir dir1 dir2
touch dir1/file{1..5}.txt

echo "=====content of dir1====="
ls dir1

echo "=====no r option====="
rsync dir1/ dir2
ls dir2

echo "=====r option====="
rsync -r dir1/ dir2
ls dir2

rm -r dir1 dir2

=====content of dir1=====
file1.txt
file2.txt
file3.txt
file4.txt
file5.txt
=====no r option=====
skipping directory .
=====r option=====
file1.txt
file2.txt
file3.txt
file4.txt
file5.txt


### `-a` - attributes

It works just like `r`, but also deals with file attributes (like creation time, user and so on). So in the following example:

- Few files was created in `dir1`;
- The user for these files has been changed from `root` to `user1`;
- Then I use `rsync` twice:
    - First with the `r` option, which puts the user back in `root`;
    - Second, with the `a` option, which saves the user attribute as `user1` from source.

In [38]:
%%bash
# docker container for usperuser access
docker run --rm --name test_container -i ubuntu
apt-get update &> /dev/null
apt-get install -y rsync &> /dev/null
# creating file with some specific user
useradd user1
mkdir dir1 dir2
touch dir1/file{1..5}.txt
chown user1 dir1/file*

echo "=====dir1====="
ls -l dir1

echo
echo "=====dir2 after rsync -r====="
rsync -r dir1/ dir2
ls -l dir2

echo
echo "=====dir2 after rsync -a====="
rsync -a dir1/ dir2
ls -l dir2

exit

=====dir1=====
total 0
-rw-r--r-- 1 user1 root 0 Jun 17 15:31 file1.txt
-rw-r--r-- 1 user1 root 0 Jun 17 15:31 file2.txt
-rw-r--r-- 1 user1 root 0 Jun 17 15:31 file3.txt
-rw-r--r-- 1 user1 root 0 Jun 17 15:31 file4.txt
-rw-r--r-- 1 user1 root 0 Jun 17 15:31 file5.txt

=====dir2 after rsync -r=====
total 0
-rw-r--r-- 1 root root 0 Jun 17 15:31 file1.txt
-rw-r--r-- 1 root root 0 Jun 17 15:31 file2.txt
-rw-r--r-- 1 root root 0 Jun 17 15:31 file3.txt
-rw-r--r-- 1 root root 0 Jun 17 15:31 file4.txt
-rw-r--r-- 1 root root 0 Jun 17 15:31 file5.txt

=====dir2 after rsync -a=====
total 0
-rw-r--r-- 1 user1 root 0 Jun 17 15:31 file1.txt
-rw-r--r-- 1 user1 root 0 Jun 17 15:31 file2.txt
-rw-r--r-- 1 user1 root 0 Jun 17 15:31 file3.txt
-rw-r--r-- 1 user1 root 0 Jun 17 15:31 file4.txt
-rw-r--r-- 1 user1 root 0 Jun 17 15:31 file5.txt


### `--delete` - delete extraneous

If you don't use this option `rsync` will save files wich are in destination directory. Оtherwise all files that are not in the source will be deleted in the destination folder. The following exampe show:
- In `dir2` I created file `test.txt`;
- First I rsync files from `dir1` to `dir2` without the `--delete` option - `test.txt` is still in `dir2`;
- Second I rsync files from `dir1` to `dir2` using the `--delete` option - `test.txt` disappears from `dir2`.

In [6]:
%%bash
mkdir dir1 dir2
touch dir1/file{1..5}.txt
touch dir2/test.txt

echo "====initial dir2====="
ls dir2

echo "=====dir2 after rsync no --delete====="
rsync -r dir1/ dir2
ls dir2

echo "=====dir2 after rsync with --delete====="
rsync -r --delete dir1/ dir2
ls dir2

rm -r dir1 dir2

====initial dir2=====
test.txt
=====dir2 after rsync no --delete=====
file1.txt
file2.txt
file3.txt
file4.txt
file5.txt
test.txt
=====dir2 after rsync with --delete=====
file1.txt
file2.txt
file3.txt
file4.txt
file5.txt
